In [1]:
#Project 3 Scratchwork Notebook
#
#
#
#This notebook crawls the reddit website. I had to adjust my crawling a few times to get the right number of posts, which is why so much of my code is commented out.
##
##
##   WARNING:
#    Do not run this notebook all the way through, it will take a long time and has a lot of printouts which I used to track my progress.

In [2]:
import requests
import pandas as pd

In [3]:
#URL to the reddit API
url = 'https://api.pushshift.io/reddit/search/submission'

In [4]:
# #Building a sample pull. Have since learned there is a 100 post limit.
 params = {
     'subreddit':'LifeProTips',
     'size':500,
     'before':1610463397
 }

Below I do a test pull, and verify the creation date of the last post.

In [15]:
#This cell I used to get the UTC time of the last post that was pulled.
res_2 = requests.get(url, params)

data_2 = res_2.json()

posts_2 = data_2['data']

df_2 = pd.DataFrame(posts_2)

df_2.iloc[-1]['created_utc']

Then I decide that a function is the best way to automate the pulling of multiple batches of posts:

In [24]:
#Maybe I should write a function to do this over and over, since I am only getting 100 posts at a time.

In [25]:
#Import and build function:
#Goal is to loop through using the 'created_utc' timestamp for n number of iterations, to get 100* that many posts.
#This can be used to greatly increase the amount of data we have access to.
#Inputs would be a subreddit name, and number of posts by 100 to select. 
#The function would make all of the requests, and append them to a list or concatenate them to a df.

Below I am testing that I can update the before parameter, so I can use it later in my loop to pull and change the date from which I am pulling:

In [26]:
p = {
    'subreddit':'LifeProTips',
    'size':500
}
p.update({'before':1000})
p

{'subreddit': 'LifeProTips', 'size': 500, 'before': 1000}

There is a lot of data associated with each post. As I want to predict based on the text or title of a post, I want to limit the data I eventually use to 'title' and 'selftext'. I feel the comments, while being able to contribute to the algorithm's ability to determine an ethical post from an unethical one, are not in the spirit of classifying the poster's intent, which is central to the ethicality of the post. I therefore will not use the comments in my model.

In [27]:
df_2.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_text', 'author_flair_text_color', 'awarders', 'banned_by',
       'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link',
       'gildings', 'id', 'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'removed_by_category', 'retrieved_on', 'score', 'send_replies',
       'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'subreddit_type', 'suggested_sort',
       'thumbnail', 'title', 'total_awards_received', 'treatment_tags',
       'upvote_ratio', 'url

## Here is my function, to build a dataframe out of n*100 posts from Reddit.

In [28]:
def build_reddit_df(subreddit, before_date, n):
    '''
    Function loops backwards through a given subreddit, 
    given a certain starting date, and returns n*100
    posts and post text, as well as the url of the post.
    #
    Inputs:
    subreddit (str)  : string name of the subreddit. Must be a valid subreddit. See reddit.com
    -
    before_date (int): UTC encoded date. Use https://www.epochconverter.com/ if necessary.
    -
    n (int)          : Number of posts desired, times 100. For example n = 3 will return 300 posts.
    -
    Returns:
    subreddit_df(pd.DataFrame): Returns a dataframe with the n*100 number of posts, post text, and other important
    data.
    '''
    #Create our holder df:
    df = pd.DataFrame(columns=['all_awardings', 'allow_live_comments', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_text', 'author_flair_text_color', 'awarders', 'banned_by',
       'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link',
       'gildings', 'id', 'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'removed_by_category', 'retrieved_on', 'score', 'send_replies',
       'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'subreddit_type', 'suggested_sort',
       'thumbnail', 'title', 'total_awards_received', 'treatment_tags',
       'upvote_ratio', 'url', 'whitelist_status', 'wls',
       'author_flair_richtext', 'author_flair_type', 'author_fullname',
       'author_patreon_flair', 'author_premium', 'link_flair_css_class',
       'link_flair_template_id', 'link_flair_text', 'selftext', 'post_hint',
       'preview'])
    #Declare the base URL of reddit requests location:
    url = 'https://api.pushshift.io/reddit/search/submission'
    #Set up the parameters:
    params = {
        'subreddit':subreddit,
        'size':100,
        'before':before_date
    }
    #Loop through n times, grabbing the data we need, and updating the start search term:
    for i in range(n):
        #Send the request and read in using json function:
        res = requests.get(url, params)
        try:
            data = res.json()
            #Data is in the 'data' column, make a temp df from it:
            posts = data['data']
            post_df = pd.DataFrame(posts)
            print(len(df))
            df = df.append(post_df)
            #Update the before date to the date of the oldest post in the group we just added:
            params.update({'before':post_df.iloc[-1]['created_utc']})
        except:
            #If you encounter an error, skip behind a day, and subtract one from i to make sure you get enough:
            params.update({'before':(df.iloc[-1]['created_utc']-86_400)})
            i-=1
    return df

In [ ]:
#UTC date was chosen at random, corresponds to 10PM Jan 12 2021

The below code is an artifact of the first pulls I did. I have left this code to show my thought process when determining how many posts to pull:

In [ ]:
#####################################################DO NOT UNCOMMENT################################################
#Ok we need to look at these posts, and see if any of them have been removed or are nan, and go back for more posts 

#if there aren't enough full posts.

#lpt_df[['selftext', 'title']].isna().value_counts()

#ulpt_df[['selftext', 'title']].isna().value_counts()

#Not too bad, lets drop the Nans.
# lpt_df.dropna(subset = ['selftext', 'title'], inplace=True)
# len(lpt_df)


# ulpt_df.dropna(subset = ['selftext', 'title'], inplace=True)
# len(ulpt_df)

#lpt_df[['selftext', 'title']].iloc[3][0]

#Ok lets see how much we would be left with when we drop the ['removed'] posts:
#len(lpt_df[lpt_df['selftext'] != '[removed]'])

#len(ulpt_df[ulpt_df['selftext'] != '[removed]'])
#####################################################################################################################

The end result, I need to pull 20_000 to 22_000 posts to get the number of posts I would like to have in the end (about 7500 posts each)

I have commented the cell out below, so it is not run in error. If you wish to run the cell, it will count out in 100s how many posts it has pulled.
This is how I tracked the progress and verified it was working as intended while I went and did other work.

In [32]:
# # #Looks like we need to double our crawling.
# lpt_df_2 = build_reddit_df('LifeProTips', 1610427600, 200)
# ulpt_df_2 = build_reddit_df('UnethicalLifeProTips', 1610427600, 220)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [1]:
#drop the nans.
lpt_df_2.dropna(subset = ['selftext', 'title'], inplace=True)
ulpt_df_2.dropna(subset = ['selftext', 'title'], inplace=True)

NameError: name 'lpt_df_2' is not defined

In [33]:
#export the dataframes to csv
lpt_df_2.to_csv('./data/lpt_df_2.csv')
ulpt_df_2.to_csv('./data/ulpt_df_2.csv')

And that's it. Data is scraped from reddit, we have removed the Nans, and will remove posts that are not applicable ('removed' posts) in a later step.